In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as st
import cufflinks as cf
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected = True)
cf.go_offline()
import chart_studio.plotly as py
import plotly.graph_objs as go 
import datetime as dt
import dataframe_image as dfi
import Levenshtein as lev
import re

In [11]:
df = pd.read_csv("GCST.csv", low_memory = False)

In [12]:
# CHANGING DECISION DATE AND APPLICATION RECEIVE DATE INTO MONTH OF YEAR 
df["MONTH"] = pd.to_datetime(df["DECISION_DATE"]).apply(lambda x:x.month)
df["YEAR"] = pd.to_datetime(df["DECISION_DATE"]).apply(lambda x:x.year)

In [13]:
# DROP RECIEVED DATE AFTER EXTRACTING MONTH OF YEAR AND SETING INDEX TO DECISION DATE
# RECEIVED DATE ISNT NEEDED WITH RECIEVE - DECISION COLUMN AND DECISION DATE
df = df.set_index("DECISION_DATE")
df = df.drop("RECEIVED_DATE", axis = 1)

In [14]:
# changing certified and certified expired to 1. and withdrawn and denied to 0: CLASSIFICATION MODEL OUTPUT
df["CASE_STATUS"] = df["CASE_STATUS"].replace(["CERTIFIED-EXPIRED","CERTIFIED"],[1,1]).replace(["WITHDRAWN","DENIED"],[0,0])

In [21]:
# Cleaning up salary offered, and salary PW: if not number then null
df["WAGE_OFFER_FROM"] = pd.to_numeric(df["WAGE_OFFER_FROM"], errors='coerce')
df["PW_WAGE"] = pd.to_numeric(df["PW_WAGE"], errors='coerce')

In [31]:
# ARE THERE ANY APPLICATIONS WITH NO WAGE OFFERED?
print("Number of Successful applicants:",df[df["WAGE_OFFER_FROM"].isnull()]["CASE_STATUS"].sum())
print("Out of:",len(df[df["WAGE_OFFER_FROM"].isnull()]),",",df[df["WAGE_OFFER_FROM"].isnull()]["CASE_STATUS"].sum()/len(df[df["WAGE_OFFER_FROM"].isnull()]))

Number of Successful applicants: 7
Out of: 158 , 0.04430379746835443


In [32]:
# ARE THERE ANY APPLICATIONS WITH NO PW?
print("Number of Successful applicants:",df[df["PW_WAGE"].isnull()]["CASE_STATUS"].sum())
print("Out of:",len(df[df["PW_WAGE"].isnull()]),",",df[df["PW_WAGE"].isnull()]["CASE_STATUS"].sum()/len(df[df["PW_WAGE"].isnull()]))

Number of Successful applicants: 408
Out of: 692 , 0.5895953757225434


In [34]:
# DROPPING ANY APPLICANTS WITHOUT A WAGE OFFER OR PW WAGE
df = df[~df["WAGE_OFFER_FROM"].isnull()]
df = df[~df["PW_WAGE"].isnull()]

In [47]:
# STANDARD THE PW and SALARY UNIT OF PAY
df["PW_UNIT_OF_PAY"] = df["PW_UNIT_OF_PAY"].replace(["HOUR","YEAR","WEEK","MONTH","BI-WEEKLY"],["HR","YR","WK","MTH","BI"])
df["WAGE_OFFER_UNIT_OF_PAY"] = df["WAGE_OFFER_UNIT_OF_PAY"].replace(["HOUR","YEAR","WEEK","MONTH","BI-WEEKLY"],["HR","YR","WK","MTH","BI"])

In [48]:
# DISTRIBUTION OF UNIT OF PAY
pd.DataFrame([df["WAGE_OFFER_UNIT_OF_PAY"].value_counts(),df["PW_UNIT_OF_PAY"].value_counts(),df["WAGE_OFFER_UNIT_OF_PAY"].value_counts()-df["PW_UNIT_OF_PAY"].value_counts()]).transpose()

,WAGE_OFFER_UNIT_OF_PAY,PW_UNIT_OF_PAY,Unnamed 0
YR,710131,723665,-13534
HR,25093,11847,13246
WK,301,152,149
MTH,233,123,110
BI,64,36,28


In [49]:
# ANNUALIZING WAGE OFFER AND PW
# Annualized Salaries: Actual Salary: multiplier
def annual(row):
    if row["WAGE_OFFER_UNIT_OF_PAY"] == "YR":
        return 1
    elif row["WAGE_OFFER_UNIT_OF_PAY"] == "MTH": 
        return 12
    elif row["WAGE_OFFER_UNIT_OF_PAY"] == "WK": 
        return 52
    elif row["WAGE_OFFER_UNIT_OF_PAY"] == "BI": 
        return 26
    elif row["WAGE_OFFER_UNIT_OF_PAY"] == "HR": 
        return 2080
mult = pd.DataFrame(df.apply(lambda row:annual(row), axis = 1))
mult.columns = ["WAGE_OFFER_FROM"]
# ANNUALIZED WAGE OFFER
df["WO_A"] = pd.DataFrame(df["WAGE_OFFER_FROM"])*mult

# Annualized Salaries: PW Salary
def annuali(row):
    if row["PW_UNIT_OF_PAY"] == "YR":
        return 1
    elif row["PW_UNIT_OF_PAY"] == "MTH": 
        return 12
    elif row["PW_UNIT_OF_PAY"] == "WK": 
        return 52
    elif row["PW_UNIT_OF_PAY"] == "BI": 
        return 26
    elif row["PW_UNIT_OF_PAY"] == "HR": 
        return 2080
multi = pd.DataFrame(df.apply(lambda row:annuali(row), axis = 1))
multi.columns = ["PW_WAGE"]
# ANNUALIZED WAGE PW
df["PW_A"] = pd.DataFrame(df["PW_WAGE"])*multi

In [51]:
# ANNUALIZED WAGE OFFER - ANNUALIZED PW
df["WO-PW"] = df["WO_A"] - df["PW_A"]

In [60]:
# HAVE WAGE OFFER BEEN < ANNUALIZED PW? THIS SOULD NEVER HAPPEN...
lowwage = df[df["WO-PW"]<0][["CASE_STATUS","WAGE_OFFER_UNIT_OF_PAY","PW_UNIT_OF_PAY","WAGE_OFFER_FROM","PW_WAGE","WO_A","PW_A"]]
lowwage

,CASE_STATUS,WAGE_OFFER_UNIT_OF_PAY,PW_UNIT_OF_PAY,WAGE_OFFER_FROM,PW_WAGE,WO_A,PW_A
DECISION_DATE,,,,,,,
10/5/2020,1,HR,YR,16.16,33613.00,33612.8,33613.0
10/6/2020,1,HR,YR,20.27,42162.00,42161.6,42162.0
10/6/2020,1,HR,YR,20.27,42162.00,42161.6,42162.0
10/7/2020,1,HR,YR,20.27,42162.00,42161.6,42162.0
10/22/2020,1,HR,YR,11.41,23733.00,23732.8,23733.0
...,...,...,...,...,...,...,...
10/28/2014,0,HR,HR,10.00,20.00,20800.0,41600.0
4/6/2015,0,HR,HR,11.50,12.00,23920.0,24960.0
10/20/2014,0,HR,HR,25.00,26.07,52000.0,54225.6


In [61]:
# HOW MANY HAVE SUCCESSFUL APPLICATIONS WHEN WAGE < PW?
print("Number of Successful applicants:",lowwage["CASE_STATUS"].sum())
print("Out of:",len(lowwage),",",lowwage["CASE_STATUS"].sum()/len(lowwage))

Number of Successful applicants: 830
Out of: 1515 , 0.5478547854785478


In [63]:
# SUCCESFULL APPLICATIONS WITH THE SAME UNIT
lowwage[lowwage["WAGE_OFFER_UNIT_OF_PAY"]==lowwage["PW_UNIT_OF_PAY"]]

,CASE_STATUS,WAGE_OFFER_UNIT_OF_PAY,PW_UNIT_OF_PAY,WAGE_OFFER_FROM,PW_WAGE,WO_A,PW_A
DECISION_DATE,,,,,,,
11/4/2020,0,YR,YR,40000.0,94078.00,40000.0,94078.0
11/9/2020,0,YR,YR,21019.0,27019.00,21019.0,27019.0
12/2/2020,0,YR,YR,27394.0,27934.00,27394.0,27934.0
12/14/2020,0,YR,YR,106038.0,128898.00,106038.0,128898.0
12/17/2020,0,YR,YR,106038.0,110677.00,106038.0,110677.0
...,...,...,...,...,...,...,...
10/28/2014,0,HR,HR,10.0,20.00,20800.0,41600.0
4/6/2015,0,HR,HR,11.5,12.00,23920.0,24960.0
10/20/2014,0,HR,HR,25.0,26.07,52000.0,54225.6


In [126]:
# PATTERN FOR MIS-LABELED: UNIT OF PAY IS NOT YEAR BUT THE WAGE OFFER IS YEAR AMOUNT
# FIND THIS BY UNIT OF PAY NOT YEAR, BUT WAGE OFFER > btm 25% OF WAGE OFFER YEAR
top = df[df["WAGE_OFFER_UNIT_OF_PAY"]=="YR"].groupby("YEAR")["WAGE_OFFER_FROM"].quantile(.10) 
top

YEAR
2014    57000.0
2015    48069.0
2016    46218.0
2017    42416.8
2018    43618.0
2019    40872.0
2020    41954.0
2021    43680.0
Name: WAGE_OFFER_FROM, dtype: float64

In [127]:
# CREATING THE LIST FOR THE FOR LOOP TO REPLACE IF ANNUALIZED SALARY IS > OUTLIER CUTOFF
year = top.index.tolist()
top = top.tolist()

In [130]:
# GRABBING THE APPLICANTS THAT ARE OVER NON YR BUT > btm 10% of WAGE
todrop = pd.DataFrame()
for i in range(len(year)):
    j = df[(df["YEAR"]==year[i])&(df["WAGE_OFFER_FROM"]>=top[i])&(df["WAGE_OFFER_UNIT_OF_PAY"]!="YR")]
    todrop = pd.concat([todrop,j])
todrop[["CASE_STATUS","WAGE_OFFER_UNIT_OF_PAY","PW_UNIT_OF_PAY","WAGE_OFFER_FROM","PW_WAGE","WO_A","PW_A"]]

,CASE_STATUS,WAGE_OFFER_UNIT_OF_PAY,PW_UNIT_OF_PAY,WAGE_OFFER_FROM,PW_WAGE,WO_A,PW_A
DECISION_DATE,,,,,,,
11/6/2014,1,NaN,NaN,258362.00,90.00,NaN,NaN
10/22/2014,1,NaN,NaN,105747.20,50.84,NaN,NaN
10/14/2014,1,NaN,NaN,106520.58,46.49,NaN,NaN
12/5/2014,1,NaN,NaN,80000.00,54642.00,NaN,NaN
11/21/2014,1,NaN,NaN,120000.00,1694.36,NaN,NaN
...,...,...,...,...,...,...,...
4/14/2021,1,WK,YR,147150.00,115361.00,7651800.0,115361.0
5/21/2021,1,WK,YR,125000.00,66331.00,6500000.0,66331.0
6/29/2021,0,HR,YR,43867.00,43867.00,91243360.0,43867.0


In [129]:
# HOW MANY OF THE APPLICANTS WITH OUTLIER WAGE WERE SUCCESFUL?
print("Number of Successful applicants:",todrop["CASE_STATUS"].sum())
print("Out of:",len(todrop),",",todrop["CASE_STATUS"].sum()/len(todrop))

Number of Successful applicants: 210
Out of: 265 , 0.7924528301886793


In [115]:
topdrop[topdrop["WO_A"] == topdrop["WO_A"].max()]

,CASE_STATUS,DAYS_TO_DECIDE,REFILE,EMPLOYER_NAME,EMPLOYER_CITY,EMPLOYER_STATE_PROVINCE,EMPLOYER_COUNTRY,EMPLOYER_NUM_EMPLOYEES,EMPLOYER_YEAR_COMMENCED_BUSINESS,PW_SOC_CODE,...,FOREIGN_WORKER_REQ_EXPERIENCE,FOREIGN_WORKER_ALT_ED_EXP,FOREIGN_WORKER_ALT_OCC_EXP,EMPLOYER_COMPLETED_APPLICATION,NAICS_CODE,MONTH,YEAR,WO_A,PW_A,WO-PW
DECISION_DATE,,,,,,,,,,,,,,,,,,,,,
6/29/2021,0,0.0,N,"TANTO IRRIGATION, LLC",ELMSFORD,NEW YORK,UNITED STATES OF AMERICA,41.0,2002.0,37-1012,...,N,NaN,Y,N,22131,6,2021,91243360.0,43867.0,91199493.0


In [ ]:
# CLEANING UP STATES

In [ ]:
# ADDING CATEGORICAL VARIABLE OF DISCREPANCY BETWEEN EMPLOYER HQ STATE AND CITY WITH JOB WORK SITE. UNIT OF WAGE AS WELL
df["DISC_ST"] = np.where(df["EMPLOYER_STATE"]==df["JOB_INFO_WORK_STATE"], 0, 1)
df["DISC_CTY"] = np.where(df["EMPLOYER_CITY"]==df["JOB_INFO_WORK_CITY"], 0, 1)
df["DISC_UNIT"] = np.where(df["WAGE_OFFER_UNIT_OF_PAY_9089"]==df["PW_UNIT_OF_PAY_9089"], 0, 1)

In [ ]:
# CLEANING UP THE CLASS OF ADMISSIONS
df.loc[df["CLASS_OF_ADMISSION"].str.contains("PAROL", case=False, na = False), "CLASS_OF_ADMISSION"] = "PAROLEE"
df["CLASS_OF_ADMISSION"] = np.where((df["CLASS_OF_ADMISSION"] == "H1B1"),"H-1B1",df["CLASS_OF_ADMISSION"])
df["CLASS_OF_ADMISSION"] = np.where((df["CLASS_OF_ADMISSION"] == "H-1B/I-485 APP"),"H-1B",df["CLASS_OF_ADMISSION"])
df["CLASS_OF_ADMISSION"] = np.where((df["CLASS_OF_ADMISSION"] == "AOS/H-1B"),"H-1B",df["CLASS_OF_ADMISSION"])
df["CLASS_OF_ADMISSION"] = np.where((df["CLASS_OF_ADMISSION"] == "H1B"),"H-1B",df["CLASS_OF_ADMISSION"])

In [ ]:
#manual Filter employers
df.loc[df["EMPLOYER_NAME"].str.contains("GOOGLE", case = False, na = False),"EMPLOYER_NAME"]="GOOGLE"
df.loc[(df["EMPLOYER_NAME"].str.contains("amazon |amazon.com", case = False, na = False))&(~df["EMPLOYER_NAME"].str.contains("amazon conservation team|amazon stones", case = False, na = False)), "EMPLOYER_NAME"] = "AMAZON"
df.loc[df["EMPLOYER_NAME"].str.contains("MICROSOFT", case = False, na = False), "EMPLOYER_NAME"] = "MICROSOFT"
df.loc[df["EMPLOYER_NAME"].str.contains("Cognizant", case = False, na = False), "EMPLOYER_NAME"] = "COGNIZANT"
df.loc[df["EMPLOYER_NAME"].str.contains("intel corporation|intel mobile|intel massachusetts|intel america|intel federal", case = False, na = False), "EMPLOYER_NAME"] = "INTEL"
df.loc[df["EMPLOYER_NAME"].str.contains("apple inc.|apple computer|apple educational", case = False, na = False), "EMPLOYER_NAME"] = "APPLE"
df.loc[df["EMPLOYER_NAME"].str.contains("infosys ltd.|infosys technologies|infosys public|infosys solutions|infosys consulting|infosys international|infosys BPO", case = False, na = False), "EMPLOYER_NAME"] = "INFOSYS"
df.loc[df["EMPLOYER_NAME"].str.contains("facebook", case = False, na = False), "EMPLOYER_NAME"] = "FACEBOOK"
df.loc[df["EMPLOYER_NAME"].str.contains("tata consultancy|tata technologies|tata communications|tata industries|tata consltancy|tata enterprises|tata america international|tata autocomp|tata international metals|tata chemicals|tata elxsi", case = False, na = False), "EMPLOYER_NAME"] = "TATA"
df.loc[df["EMPLOYER_NAME"].str.contains("oracle america|oracle usa|oracle financial", case = False, na = False), "EMPLOYER_NAME"] = "FACEBOOK"
df.loc[df["EMPLOYER_NAME"].str.contains("hcl america", case = False, na = False), "EMPLOYER_NAME"] = "HCL AMERICA"
df.loc[df["EMPLOYER_NAME"].str.contains("deloitte", case = False, na = False), "EMPLOYER_NAME"] = "DELOITTE"
df.loc[df["EMPLOYER_NAME"].str.contains("qualcomm", case = False, na = False), "EMPLOYER_NAME"] = "QUALCOMM"
df.loc[df["EMPLOYER_NAME"].str.contains("IBM ", case = False, na = False), "EMPLOYER_NAME"] = "IBM"
df.loc[df["EMPLOYER_NAME"].str.contains("Cisco systems|cisco consumer", case = False, na = False), "EMPLOYER_NAME"] = "CISCO"
df.loc[df["EMPLOYER_NAME"].str.contains("salesforce", case = False, na = False), "EMPLOYER_NAME"] = "SALESFORCE"
df.loc[df["EMPLOYER_NAME"].str.contains("morgan stanley", case = False, na = False), "EMPLOYER_NAME"] = "MORGAN STANLEY"
df.loc[df["EMPLOYER_NAME"].str.contains("JP Morgan", case = False, na = False), "EMPLOYER_NAME"] = "JP MORGAN"
df.loc[df["EMPLOYER_NAME"].str.contains("goldman sachs|goldman, sachs|goldman,sachs", case = False, na = False), "EMPLOYER_NAME"] = "GOLDMAN SACHS"
df.loc[df["EMPLOYER_NAME"].str.contains("MERRILL LYNCH", case = False, na = False), "EMPLOYER_NAME"] = "MERRILL LYNCH"
df.loc[df["EMPLOYER_NAME"].str.contains("Citibank|citigroup", case = False, na = False), "EMPLOYER_NAME"] = "CITI GROUP"
df.loc[df["EMPLOYER_NAME"].str.contains("CREDIT SUISSE", case = False, na = False), "EMPLOYER_NAME"] = "CREDIT SUISSE"
df.loc[df["EMPLOYER_NAME"].str.contains("pricewater|pwc", case = False, na = False), "EMPLOYER_NAME"] = "PRICEWATERHOUSECOOPERS"
df.loc[df["EMPLOYER_NAME"].str.contains("ernst  |ernst & Young", case = False, na = False), "EMPLOYER_NAME"] = "ERNST YOUNG"
df.loc[df["EMPLOYER_NAME"].str.contains("capgemini", case = False, na = False), "EMPLOYER_NAME"] = "CAPGEMINI"
df.loc[df["EMPLOYER_NAME"].str.contains("technip usa", case = False, na = False), "EMPLOYER_NAME"] = "TECHNIP USA"
df.loc[df["EMPLOYER_NAME"].str.contains("bechtel ", case = False, na = False), "EMPLOYER_NAME"] = "BECHTEL"
df.loc[(df["EMPLOYER_NAME"].str.contains("hp enterprise services|hp inc.|Hewlett-|Hewlett P|HewlettP", case = False, na = False))&(~df["EMPLOYER_NAME"].str.contains("queen nails hp inc|backer ehp inc", case = False, na = False)), "EMPLOYER_NAME"] = "AMAZON"
df.loc[df["EMPLOYER_NAME"].str.contains("CGGVERITAS", case = False, na = False), "EMPLOYER_NAME"] = "CGGVERITAS"
df.loc[df["EMPLOYER_NAME"].str.contains("national oilwell", case = False, na = False), "EMPLOYER_NAME"] = "NATIONAL OILWELL VARCO"
df.loc[df["EMPLOYER_NAME"].str.contains("IH SERVICES", case = False, na = False), "EMPLOYER_NAME"] = "IH SERVICES"
df.loc[df["EMPLOYER_NAME"].str.contains("paypal", case = False, na = False), "EMPLOYER_NAME"] = "PAYPAL"
df.loc[df["EMPLOYER_NAME"].str.contains("EBAY INC.|EBAY INC|EBAY ENTERPRISE|EBAY MOBILE LABS| EBAY PAYMENTS", case = False, na = False), "EMPLOYER_NAME"] = "EBAY"
df.loc[df["EMPLOYER_NAME"].str.contains("brocade", case = False, na = False), "EMPLOYER_NAME"] = "BROCADE"
df.loc[df["EMPLOYER_NAME"].str.contains("ADOBE INC.|ADOBE SYSTEMS", case = False, na = False), "EMPLOYER_NAME"] = "ADOBE"
df.loc[df["EMPLOYER_NAME"].str.contains("University of Chicago", case = False, na = False), "EMPLOYER_NAME"] = "THE UNIVERSITY OF CHICAGO"
df.loc[df["EMPLOYER_NAME"].str.contains("mckinsey", case = False, na = False), "EMPLOYER_NAME"] = "MCKINSEY COMPANY"
df.loc[df["EMPLOYER_NAME"].str.contains("motorola", case = False, na = False), "EMPLOYER_NAME"] = "MOTOROLA"
df.loc[df["EMPLOYER_NAME"].str.contains("citadel LLC|citadel americas|citadel Securities|citadel investment|citadel enterprise", case = False, na = False), "EMPLOYER_NAME"] = "CITADEL LLC"
df.loc[df["EMPLOYER_NAME"].str.contains("yahoo", case = False, na = False), "EMPLOYER_NAME"] = "YAHOO"
df.loc[df["EMPLOYER_NAME"].str.contains("Juniper networks", case = False, na = False), "EMPLOYER_NAME"] = "JUNIPER NETWORKS"
df.loc[df["EMPLOYER_NAME"].str.contains("a2z development", case = False, na = False), "EMPLOYER_NAME"] = "A2Z DEVELOPMENT"
df.loc[df["EMPLOYER_NAME"].str.contains("walmart|wal-mart", case = False, na = False), "EMPLOYER_NAME"] = "WALMART"
df.loc[df["EMPLOYER_NAME"].str.contains("fortinet", case = False, na = False), "EMPLOYER_NAME"] = "FORTINET"
df.loc[df["EMPLOYER_NAME"].str.contains("larsen &|larsen  ", case = False, na = False), "EMPLOYER_NAME"] = "LARSEN TOUBRO"
df.loc[df["EMPLOYER_NAME"].str.contains("mindtree", case = False, na = False), "EMPLOYER_NAME"] = "MINDTREE LIMITED"
df.loc[df["EMPLOYER_NAME"].str.contains("igate tech|igate mastech|igate global", case = False, na = False), "EMPLOYER_NAME"] = "IGATE TECHNOLOGIES"
df.loc[df["EMPLOYER_NAME"].str.contains("3i info", case = False, na = False), "EMPLOYER_NAME"] = "3I INFOTECH"
df.loc[df["EMPLOYER_NAME"].str.contains("NTT Data", case = False, na = False), "EMPLOYER_NAME"] = "NTT DATA"
df.loc[df["EMPLOYER_NAME"].str.contains("Texas instrument", case = False, na = False), "EMPLOYER_NAME"] = "TEXAS INSTRUMENTS"
df.loc[df["EMPLOYER_NAME"].str.contains("tech mahindra", case = False, na = False), "EMPLOYER_NAME"] = "TECH MAHINDRA"
df.loc[df["EMPLOYER_NAME"].str.contains("DELL USA LP|DELL PRODUCTS LP|DELL MARKETING LP|DELL SOFTWARE INC|DELL FINANCIAL", case = False, na = False), "EMPLOYER_NAME"] = "DELL"
df.loc[df["EMPLOYER_NAME"].str.contains("uber technologies", case = False, na = False), "EMPLOYER_NAME"] = "UBER TECHNOLOGIES"
df.loc[df["EMPLOYER_NAME"].str.contains("case farms", case = False, na = False), "EMPLOYER_NAME"] = "CASE FARMS"
df.loc[df["EMPLOYER_NAME"].str.contains("house of raeford", case = False, na = False), "EMPLOYER_NAME"] = "HOUSE OF RAEFORD FARMS"
df.loc[df["EMPLOYER_NAME"].str.contains("george's processing", case = False, na = False), "EMPLOYER_NAME"] = "GEORGE'S PROCESSING"
df.loc[df["EMPLOYER_NAME"].str.contains("abbyland", case = False, na = False), "EMPLOYER_NAME"] = "ABBYLAND FOODS"
df.loc[df["EMPLOYER_NAME"].str.contains("koch foods of cinc", case = False, na = False), "EMPLOYER_NAME"] = "KOCH FOODS OF CINCINNATI"
df.loc[df["EMPLOYER_NAME"].str.contains("koch foods of al", case = False, na = False), "EMPLOYER_NAME"] = "KOCH FOODS OF ALABAMA"

In [ ]:
#EMPLOYER LOOK UP TOOL
#df[df["EMPLOYER_NAME"].str.contains("wayne farms", case = False, na = False)]["EMPLOYER_NAME"].value_counts()

In [ ]:
############################################################

In [ ]:
########## EDA ##########

In [ ]:
### EMPLOYERS ###

In [ ]:
# Top 15 Employers of all time 2007-2021
allemp = pd.DataFrame(df["EMPLOYER_NAME"].value_counts()[:15]).reset_index()
allemp.columns = ["AEMP","COUNT AEMP"]
sucemp = pd.DataFrame(df[df["CASE_STATUS"]==1]["EMPLOYER_NAME"].value_counts()[:15]).reset_index()
sucemp.columns = ["SEMP","COUNT SEMP"]
pd.concat([allemp,sucemp], axis = 1)

In [ ]:
# Grabbing the top 15 for each year employers: success only
def grab (year):
    return df[(df["YEAR"]==year)&(df["CASE_STATUS"]==1)]["EMPLOYER_NAME"].value_counts()[:15]
top15semp = pd.DataFrame([grab(2007),grab(2008),grab(2009),grab(2010),
             grab(2011),grab(2012),grab(2013),grab(2014),grab(2015),
             grab(2016),grab(2017),grab(2018),grab(2019),grab(2020),
             grab(2021)], index = [2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,
                                  2018,2019,2020,2021]).transpose()
# King of Consistency: Count the N in top 15 (success)
kingcons = top15semp.isnull().replace(False,0).replace(True,1)
kingcons["Yrs not in top15"] = kingcons[2007]+kingcons[2008]+kingcons[2009]+kingcons[2010]+kingcons[2011]+kingcons[2012]+kingcons[2013]+kingcons[2014]+kingcons[2015]+kingcons[2016]+kingcons[2017]+kingcons[2018]+kingcons[2019]+kingcons[2020]+kingcons[2021]
kingcons["Yrs not in top15"].sort_values()

In [ ]:
# Grabbing the top 15 for each year employers: regardless of success
def grab (year):
    return df[df["YEAR"]==year]["EMPLOYER_NAME"].value_counts()[:15]
top15aemp = pd.DataFrame([grab(2007),grab(2008),grab(2009),grab(2010),
             grab(2011),grab(2012),grab(2013),grab(2014),grab(2015),
             grab(2016),grab(2017),grab(2018),grab(2019),grab(2020),
             grab(2021)], index = [2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,
                                  2018,2019,2020,2021]).transpose()
# King of Consistency: Count the N in top 15 (application)
kingcon = top15aemp.isnull().replace(False,0).replace(True,1)
kingcon["Yrs not in top15"] = kingcon[2007]+kingcon[2008]+kingcon[2009]+kingcon[2010]+kingcon[2011]+kingcon[2012]+kingcon[2013]+kingcon[2014]+kingcon[2015]+kingcon[2016]+kingcon[2017]+kingcon[2018]+kingcon[2019]+kingcon[2020]+kingcon[2021]
kingcon["Yrs not in top15"].sort_values()

In [ ]:
### GENERAL TRENDS IN APPLICATION SUCCESS AND VOLUME ###

In [ ]:
# Denial Applications BY YEAR
ddf = pd.DataFrame(df.groupby("YEAR")["CASE_STATUS"].value_counts()[1::2])
ddf.columns = ["DENIAL"]
ddf = ddf.reset_index()
ddf = ddf.drop("CASE_STATUS", axis = 1)
ddf = ddf.set_index("YEAR")
#TOTAL APPLICATIONS BY YEAR
tap = pd.DataFrame(df.groupby("YEAR")["CASE_STATUS"].count())
tap.columns = ["TOTAL APP"]
pd.concat([ddf,tap], axis = 1).plot()
# SUCCESS APPLICATIONS BY YEAR
adf = pd.DataFrame(df.groupby("YEAR")["CASE_STATUS"].value_counts()[0::2])
adf.columns = ["ACCEPT"]
adf = adf.reset_index()
adf = adf.drop("CASE_STATUS", axis = 1)
adf = adf.set_index("YEAR")

In [ ]:
# Acceptance Correlations
st.linregress(x = adf.reset_index()["YEAR"],y = adf.reset_index()["ACCEPT"])

In [ ]:
# Acceptance as %
adf["%A"] = adf["ACCEPT"]/tap["TOTAL APP"]
adf["%A"].plot()

In [ ]:
# CORREL AND LIN REG BETWEEN % SUCCESS AND TIME
st.linregress(x = adf.reset_index()["YEAR"],y = adf.reset_index()["%A"])

In [ ]:
# DATATABLE WITH ACCEPT AND DENIAL % OVER THE YEARS
acy = df.groupby("YEAR")["CASE_STATUS"].value_counts()
acyper = pd.DataFrame(acy.groupby(level=0).transform(lambda x: (x / x.sum()).round(2)))
acyper.columns = ["Percent"]
f = acyper.reset_index()["YEAR"].tolist()
t = acyper.reset_index()["CASE_STATUS"].replace(1,"Accept").replace(0,"Reject").tolist()
idx = []
idx.extend([tuple(a) for a in zip (f,t)])
acyper.index = pd.MultiIndex.from_tuples(idx)
acyper

In [ ]:
#### WAGE ANALYSIS ####

In [ ]:
yr = df["YEAR"].unique().tolist()
yr = yr.sort()

In [ ]:
# CORREL AND LIN REG BETWEEN Wage Offered and the Year (case by case)
st.linregress(y = df.reset_index().dropna()["WO_A"], x = yr)

In [ ]:
# CORREL AND LIN REG BETWEEN Wage offered (btm 25%) and Year (in aggregate)
st.linregress(y = df.groupby("YEAR")["WO_A"].quantile(.25), x = yr)

In [ ]:
# CORREL AND LIN REG BETWEEN Wage offered (median) and Year (in aggregate)
st.linregress(y = df.groupby("YEAR")["WO_A"].quantile(.5), x = yr)

In [ ]:
# CORREL AND LIN REG BETWEEN Wage offered (top 75%) and Year (in aggregate)
st.linregress(y = df.groupby("YEAR")["WO_A"].quantile(.75), x = yr)

In [ ]:
# CORREL AND LIN REG BETWEEN WAGE OFFERED - PW WAGE and Acceptance (case by case) 
# FIRST TURN INTO A DUMMY AND THEN COMPARE WITH CASE STATUS AND COMPARE IT WITH IF WAGE OFFER NOT BIGGER: ANY DIFFERENCE?
st.linregress(x = np.where(df["OW-PW"]> 0, 1, 0),y = df["CASE_STATUS"])
st.linregress(x = np.where(df["OW-PW"]< 0, 1, 0),y = df["CASE_STATUS"])

In [ ]:
#PLOT OF AVERAGE WAGE - PW
fig,axes = plt.subplots(ncols = 2, figsize = (12,10))
df.groupby("YEAR")["OW-PW"].mean().plot(ax = axes[0],kind = "bar")
df.groupby("YEAR")["OW-PW"].quantile(.5).plot(ax = axes[1],kind = "bar")

In [ ]:
### CITY AND STATE TO WORK IN ###

In [ ]:
# ACCEPT % BASED ON STATE HQ IN

empy = df.groupby("EMPLOYER_STATE")["CASE_STATUS"].value_counts()
empyer = pd.DataFrame(empy.groupby(level=0).transform(lambda x: (x / x.sum()).round(2)))
empyer.columns = ["Percent"]
f1 = empyer.reset_index()["EMPLOYER_STATE"].tolist()
t1 = empyer.reset_index()["CASE_STATUS"].replace(1,"Accept").replace(0,"Reject").tolist()
idx1 = []
idx1.extend([tuple(a) for a in zip (f1,t1)])
empyer.index = pd.MultiIndex.from_tuples(idx1)
empyer = empyer.reset_index()
empyer = empyer[empyer["level_1"]=="Accept"]
empyer = empyer.drop("level_1",axis = 1)

empycount = pd.DataFrame(df.groupby("EMPLOYER_STATE")["CASE_STATUS"].value_counts())
empycount.columns = ["Count"]
empycount = empycount.reset_index()
empycount = empycount[empycount["CASE_STATUS"]==1]
empycount = empycount["Count"]

empyer = pd.concat([empyer, empycount], axis = 1)
empyer.columns = ["State","Percent","Count"]
empyer["Count"] = empyer["State"].astype(str)+": "+empyer["Count"].astype(str)+" Successful Applications"
empyer

In [ ]:
# Choropleth Map of Employer State Accept %
data = dict(type = "choropleth",
           locations = empyer["State"],
           locationmode = "USA-states",
           z = empyer["Percent"].astype(float),
           text = empyer["Count"],
           colorbar={"title":"Percent Accepted"},
           colorscale = "delta",
           marker = dict(line = dict(color = "rgb(255,255,255)")) )
layout = dict(title = "Percent Accepted in Employer Headquartered States",
              geo = dict(scope = "usa",showlakes = True, lakecolor = "rgb(255,255,255)"))
choro = go.Figure(data, layout)
iplot(choro)

In [ ]:
# ACCEPT % BASED ON STATE WORKED IN

joby = df.groupby("JOB_INFO_WORK_STATE")["CASE_STATUS"].value_counts()
jobyer = pd.DataFrame(joby.groupby(level=0).transform(lambda x: (x / x.sum()).round(2)))
jobyer.columns = ["Percent"]
f1 = jobyer.reset_index()["JOB_INFO_WORK_STATE"].tolist()
t1 = jobyer.reset_index()["CASE_STATUS"].replace(1,"Accept").replace(0,"Reject").tolist()
idx1 = []
idx1.extend([tuple(a) for a in zip (f1,t1)])
jobyer.index = pd.MultiIndex.from_tuples(idx1)
jobyer = jobyer.reset_index()
jobyer = jobyer[jobyer["level_1"]=="Accept"]
jobyer = jobyer.drop("level_1",axis = 1)

jobycount = pd.DataFrame(df.groupby("JOB_INFO_WORK_STATE")["CASE_STATUS"].value_counts())
jobycount.columns = ["Count"]
jobycount = jobycount.reset_index()
jobycount = jobycount[jobycount["CASE_STATUS"]==1]
jobycount = jobycount["Count"]

jobyer = pd.concat([jobyer, jobycount], axis = 1)
jobyer.columns = ["State","Percent","Count"]
jobyer["Count"] = jobyer["State"].astype(str)+": "+jobyer["Count"].astype(str)+" Successful Applications"
jobyer

In [ ]:
# Choropleth Map of JOB State Accept %
data = dict(type = "choropleth",
           locations = jobyer["State"],
           locationmode = "USA-states",
           z = jobyer["Percent"].astype(float),
           text = jobyer["Count"],
           colorbar={"title":"Percent Accepted"},
           colorscale = "delta",
           marker = dict(line = dict(color = "rgb(255,255,255)")) )
layout = dict(title = "Percent Accepted in States Worked In",
              geo = dict(scope = "usa",showlakes = True, lakecolor = "rgb(255,255,255)"))
choro = go.Figure(data, layout)
iplot(choro)

In [ ]:
# top 15 CITY TO WORK IN (APPLICATION)
def grab (year):
    return df[df["YEAR"]==year]["JOB_INFO_WORK_CITY"].value_counts()[:15]
top15jcityapp = pd.DataFrame([grab(2007),grab(2008),grab(2009),grab(2010),
             grab(2011),grab(2012),grab(2013),grab(2014),grab(2015),
             grab(2016),grab(2017),grab(2018),grab(2019),grab(2020),
             grab(2021)], index = [2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,
                                  2018,2019,2020,2021]).transpose()
# King of Consistency: Count the N in top 15 (application) Job city
kingconjc = top15jcityapp.isnull().replace(False,0).replace(True,1)
kingconjc["Yrs not in top15"] = kingconjc[2007]+kingconjc[2008]+kingconjc[2009]+kingconjc[2010]+kingconjc[2011]+kingconjc[2012]+kingconjc[2013]+kingconjc[2014]+kingconjc[2015]+kingconjc[2016]+kingconjc[2017]+kingconjc[2018]+kingconjc[2019]+kingconjc[2020]+kingconjc[2021]
kingconjc["Yrs not in top15"].sort_values()

In [ ]:
# Top 15 City TO WORK IN (SUCCESS)
def grab (year):
    return df[(df["YEAR"]==year)&(df["CASE_STATUS"]==1)]["JOB_INFO_WORK_CITY"].value_counts()[:15]
top15jcitysuc = pd.DataFrame([grab(2007),grab(2008),grab(2009),grab(2010),
             grab(2011),grab(2012),grab(2013),grab(2014),grab(2015),
             grab(2016),grab(2017),grab(2018),grab(2019),grab(2020),
             grab(2021)], index = [2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,
                                  2018,2019,2020,2021]).transpose()
# King of Consistency: Count the N in top 15 (success) Job city
kingconjc = top15jcitysuc.isnull().replace(False,0).replace(True,1)
kingconjc["Yrs not in top15"] = kingconjc[2007]+kingconjc[2008]+kingconjc[2009]+kingconjc[2010]+kingconjc[2011]+kingconjc[2012]+kingconjc[2013]+kingconjc[2014]+kingconjc[2015]+kingconjc[2016]+kingconjc[2017]+kingconjc[2018]+kingconjc[2019]+kingconjc[2020]+kingconjc[2021]
kingconjc["Yrs not in top15"].sort_values()

In [ ]:
##### DEEPER DIVE INTO TOP CITY ####

In [ ]:
# Working in New York City
# TOP JOB CODES in NEW YORK CITY
appcol = pd.DataFrame(df[df["JOB_INFO_WORK_CITY"]=="NEW YORK"]["PW_SOC_CODE"].value_counts()[:15]).reset_index()
appcol.columns = ["ACode","ACount"]
succol = pd.DataFrame(df[(df["JOB_INFO_WORK_CITY"]=="NEW YORK")&(df["CASE_STATUS"]==1)]["PW_SOC_CODE"].value_counts()[:15]).reset_index()
succol.columns = ["SCode","SCount"]
pd.concat([appcol,succol], axis = 1)

In [ ]:
# Working in New York City
# EMPLOYER NAMES
appcol = pd.DataFrame(df[df["JOB_INFO_WORK_CITY"]=="NEW YORK"]["EMPLOYER_NAME"].value_counts()[:15]).reset_index()
appcol.columns = ["AName","ACount"]
succol = pd.DataFrame(df[(df["JOB_INFO_WORK_CITY"]=="NEW YORK")&(df["CASE_STATUS"]==1)]["EMPLOYER_NAME"].value_counts()[:15]).reset_index()
succol.columns = ["SName","SCount"]
pd.concat([appcol,succol], axis = 1)

In [ ]:
# Working in New York City
# Wage Offered
print("25th Percentile Application:",round(df[df["JOB_INFO_WORK_CITY"]=="NEW YORK"]["WAGE_OFFER_FROM_9089"].quantile(.25),2))
print("25th Percentile Success:",round(df[(df["JOB_INFO_WORK_CITY"]=="NEW YORK")&(df["CASE_STATUS"]==1)]["WAGE_OFFER_FROM_9089"].quantile(.25),2))
print("Mean Application:",round(df[df["JOB_INFO_WORK_CITY"]=="NEW YORK"]["WAGE_OFFER_FROM_9089"].mean(),2))
print("Mean Success:",round(df[(df["JOB_INFO_WORK_CITY"]=="NEW YORK")&(df["CASE_STATUS"]==1)]["WAGE_OFFER_FROM_9089"].mean(),2))
print("Median Application:",round(df[df["JOB_INFO_WORK_CITY"]=="NEW YORK"]["WAGE_OFFER_FROM_9089"].median(),2))
print("Median Sucess:",round(df[(df["JOB_INFO_WORK_CITY"]=="NEW YORK")&(df["CASE_STATUS"]==1)]["WAGE_OFFER_FROM_9089"].median(),2))
print("75th Percentile Application:",round(df[df["JOB_INFO_WORK_CITY"]=="NEW YORK"]["WAGE_OFFER_FROM_9089"].quantile(.75),2))
print("75th Percentile Success:",round(df[(df["JOB_INFO_WORK_CITY"]=="NEW YORK")&(df["CASE_STATUS"]==1)]["WAGE_OFFER_FROM_9089"].quantile(.75),2))

In [ ]:
# Working in New York City
# Class of Admission
appcol = pd.DataFrame(df[df["JOB_INFO_WORK_CITY"]=="NEW YORK"]["CLASS_OF_ADMISSION"].value_counts()[:15]).reset_index()
appcol.columns = ["AClass","ACount"]
succol = pd.DataFrame(df[(df["JOB_INFO_WORK_CITY"]=="NEW YORK")&(df["CASE_STATUS"]==1)]["CLASS_OF_ADMISSION"].value_counts()[:15]).reset_index()
succol.columns = ["SClass","SCount"]
pd.concat([appcol,succol], axis = 1)

In [ ]:
# Working in New York City
# Top Years
appcol = pd.DataFrame(df[df["JOB_INFO_WORK_CITY"]=="NEW YORK"]["YEAR"].value_counts()[:15]).reset_index()
appcol.columns = ["AYear","ACount"]
succol = pd.DataFrame(df[(df["JOB_INFO_WORK_CITY"]=="NEW YORK")&(df["CASE_STATUS"]==1)]["YEAR"].value_counts()[:15]).reset_index()
succol.columns = ["SYear","SCount"]
pd.concat([appcol,succol], axis = 1)

In [ ]:
# Working in New York City
# CORREL AND LIN REG BETWEEN YEARS AND NEW YORK CITY

In [ ]:
# Top 15 State worked in (application)
def grab (year):
    return df[df["YEAR"]==year]["JOB_INFO_WORK_STATE"].value_counts()[:15]
top15jstateapp = pd.DataFrame([grab(2007),grab(2008),grab(2009),grab(2010),
             grab(2011),grab(2012),grab(2013),grab(2014),grab(2015),
             grab(2016),grab(2017),grab(2018),grab(2019),grab(2020),
             grab(2021)], index = [2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,
                                  2018,2019,2020,2021]).transpose()
# King of Consistency: Count the N in top 15 (application) Job state
kingconjc = top15jstateapp.isnull().replace(False,0).replace(True,1)
kingconjc["Yrs not in top15"] = kingconjc[2007]+kingconjc[2008]+kingconjc[2009]+kingconjc[2010]+kingconjc[2011]+kingconjc[2012]+kingconjc[2013]+kingconjc[2014]+kingconjc[2015]+kingconjc[2016]+kingconjc[2017]+kingconjc[2018]+kingconjc[2019]+kingconjc[2020]+kingconjc[2021]
kingconjc["Yrs not in top15"].sort_values()

In [ ]:
# Top 15 State worked in (success)
def grab (year):
    return df[(df["YEAR"]==year)&(df["CASE_STATUS"]==1)]["JOB_INFO_WORK_STATE"].value_counts()[:15]
top15jstatesuc = pd.DataFrame([grab(2007),grab(2008),grab(2009),grab(2010),
             grab(2011),grab(2012),grab(2013),grab(2014),grab(2015),
             grab(2016),grab(2017),grab(2018),grab(2019),grab(2020),
             grab(2021)], index = [2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,
                                  2018,2019,2020,2021]).transpose()
# King of Consistency: Count the N in top 15 (success) Job state
kingconjc = top15jstatesuc.isnull().replace(False,0).replace(True,1)
kingconjc["Yrs not in top15"] = kingconjc[2007]+kingconjc[2008]+kingconjc[2009]+kingconjc[2010]+kingconjc[2011]+kingconjc[2012]+kingconjc[2013]+kingconjc[2014]+kingconjc[2015]+kingconjc[2016]+kingconjc[2017]+kingconjc[2018]+kingconjc[2019]+kingconjc[2020]+kingconjc[2021]
kingconjc["Yrs not in top15"].sort_values()

In [ ]:
##### DEEPER DIVE INTO TOP STATE ####

In [ ]:
# Working in CA State
# TOP JOB CODES in CA 
appcol = pd.DataFrame(df[df["JOB_INFO_WORK_STATE"]=="CA"]["PW_SOC_CODE"].value_counts()[:15]).reset_index()
appcol.columns = ["ACode","ACount"]
succol = pd.DataFrame(df[(df["JOB_INFO_WORK_STATE"]=="CA")&(df["CASE_STATUS"]==1)]["PW_SOC_CODE"].value_counts()[:15]).reset_index()
succol.columns = ["SCode","SCount"]
pd.concat([appcol,succol], axis = 1)

In [ ]:
# Working in CA State
# EMPLOYER NAMES
appcol = pd.DataFrame(df[df["JOB_INFO_WORK_STATE"]=="CA"]["EMPLOYER_NAME"].value_counts()[:15]).reset_index()
appcol.columns = ["AName","ACount"]
succol = pd.DataFrame(df[(df["JOB_INFO_WORK_STATE"]=="CA")&(df["CASE_STATUS"]==1)]["EMPLOYER_NAME"].value_counts()[:15]).reset_index()
succol.columns = ["SName","SCount"]
pd.concat([appcol,succol], axis = 1)

In [ ]:
# Working in CA State
# Wage Offered
print("25th Percentile Application:",round(df[df["JOB_INFO_WORK_STATE"]=="CA"]["WAGE_OFFER_FROM_9089"].quantile(.25),2))
print("25th Percentile Success:",round(df[(df["JOB_INFO_WORK_STATE"]=="CA")&(df["CASE_STATUS"]==1)]["WAGE_OFFER_FROM_9089"].quantile(.25),2))
print("Mean Application:",round(df[df["JOB_INFO_WORK_STATE"]=="CA"]["WAGE_OFFER_FROM_9089"].mean(),2))
print("Mean Success:",round(df[(df["JOB_INFO_WORK_STATE"]=="CA")&(df["CASE_STATUS"]==1)]["WAGE_OFFER_FROM_9089"].mean(),2))
print("Median Application:",round(df[df["JOB_INFO_WORK_STATE"]=="CA"]["WAGE_OFFER_FROM_9089"].median(),2))
print("Median Sucess:",round(df[(df["JOB_INFO_WORK_STATE"]=="CA")&(df["CASE_STATUS"]==1)]["WAGE_OFFER_FROM_9089"].median(),2))
print("75th Percentile Application:",round(df[df["JOB_INFO_WORK_STATE"]=="CA"]["WAGE_OFFER_FROM_9089"].quantile(.75),2))
print("75th Percentile Success:",round(df[(df["JOB_INFO_WORK_STATE"]=="CA")&(df["CASE_STATUS"]==1)]["WAGE_OFFER_FROM_9089"].quantile(.75),2))

In [ ]:
# Working in CA State
# Class of Admission
appcol = pd.DataFrame(df[df["JOB_INFO_WORK_STATE"]=="CA"]["CLASS_OF_ADMISSION"].value_counts()[:15]).reset_index()
appcol.columns = ["AClass","ACount"]
succol = pd.DataFrame(df[(df["JOB_INFO_WORK_STATE"]=="CA")&(df["CASE_STATUS"]==1)]["CLASS_OF_ADMISSION"].value_counts()[:15]).reset_index()
succol.columns = ["SClass","SCount"]
pd.concat([appcol,succol], axis = 1)

In [ ]:
# Working in CA State
# Top Years
appcol = pd.DataFrame(df[df["JOB_INFO_WORK_STATE"]=="CA"]["YEAR"].value_counts()[:15]).reset_index()
appcol.columns = ["AYear","ACount"]
succol = pd.DataFrame(df[(df["JOB_INFO_WORK_STATE"]=="CA")&(df["CASE_STATUS"]==1)]["YEAR"].value_counts()[:15]).reset_index()
succol.columns = ["SYear","SCount"]
pd.concat([appcol,succol], axis = 1)

In [ ]:
# Working in CA State
# CORREL AND LIN REG BETWEEN YEARS AND CA 

In [ ]:
#### JOB CODE ANALYSIS #####

In [ ]:
# TOP 15 JOB CODES
allcode = pd.DataFrame(df["PW_SOC_CODE"].value_counts()[:15]).reset_index()
allcode.columns = ["ACODE","COUNT ACODE"]
succode = pd.DataFrame(df[df["CASE_STATUS"]==1]["PW_SOC_CODE"].value_counts()[:15]).reset_index()
succode.columns = ["SCODE","COUNT SCODE"]
pd.concat([allcode,succode], axis = 1)

In [ ]:
# TOP 15 Apps codes
def grab (year):
    return df[df["YEAR"]==year]["PW_SOC_CODE"].value_counts()[:15]
top15code = pd.DataFrame([grab(2007),grab(2008),grab(2009),grab(2010),
             grab(2011),grab(2012),grab(2013),grab(2014),grab(2015),
             grab(2016),grab(2017),grab(2018),grab(2019),grab(2020),
             grab(2021)], index = [2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,
                                  2018,2019,2020,2021]).transpose()
# King of Consistency: Count the N in top 15 (application)
kingconjc = top15code.isnull().replace(False,0).replace(True,1)
kingconjc["Yrs not in top15"] = kingconjc[2007]+kingconjc[2008]+kingconjc[2009]+kingconjc[2010]+kingconjc[2011]+kingconjc[2012]+kingconjc[2013]+kingconjc[2014]+kingconjc[2015]+kingconjc[2016]+kingconjc[2017]+kingconjc[2018]+kingconjc[2019]+kingconjc[2020]+kingconjc[2021]
kingconjc["Yrs not in top15"].sort_values()

In [ ]:
# TOP 15 Success Codes
def grab (year):
    return df[(df["YEAR"]==year)&(df["CASE_STATUS"]==1)]["PW_SOC_CODE"].value_counts()[:15]
top15acode = pd.DataFrame([grab(2007),grab(2008),grab(2009),grab(2010),
             grab(2011),grab(2012),grab(2013),grab(2014),grab(2015),
             grab(2016),grab(2017),grab(2018),grab(2019),grab(2020),
             grab(2021)], index = [2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,
                                  2018,2019,2020,2021]).transpose()
# King of Consistency: Count the N in top 15 (success)
kingconsjc = top15acode.isnull().replace(False,0).replace(True,1)
kingconsjc["Yrs not in top15"] = kingconsjc[2007]+kingconsjc[2008]+kingconsjc[2009]+kingconsjc[2010]+kingconsjc[2011]+kingconsjc[2012]+kingconsjc[2013]+kingconsjc[2014]+kingconsjc[2015]+kingconsjc[2016]+kingconsjc[2017]+kingconsjc[2018]+kingconsjc[2019]+kingconsjc[2020]+kingconsjc[2021]
kingconsjc["Yrs not in top15"].sort_values()

In [ ]:
##### DEEPER DIVE INTO TOP 3 JOB CODE ####

In [ ]:
# 11-3021
# Computer and Information Systems Managers
# 17-2072
# Electronics Engineers, Except Computer
# 17-2141
# Mechanical Engineers

In [ ]:
# 11-3021, Computer and Information Systems Managers

# TOP CITIES WORKED IN
appcol = pd.DataFrame(df[df["PW_SOC_CODE"]=="11-3021"]["JOB_INFO_WORK_CITY"].value_counts()[:15]).reset_index()
appcol.columns = ["ACity","ACount"]
succol = pd.DataFrame(df[(df["PW_SOC_CODE"]=="11-3021")&(df["CASE_STATUS"]==1)]["JOB_INFO_WORK_CITY"].value_counts()[:15]).reset_index()
succol.columns = ["SCity","SCount"]
pd.concat([appcol,succol], axis = 1)

In [ ]:
# TOP STATES WORKED IN 
appcol = pd.DataFrame(df[df["PW_SOC_CODE"]=="11-3021"]["JOB_INFO_WORK_STATE"].value_counts()[:15]).reset_index()
appcol.columns = ["AState","ACount"]
succol = pd.DataFrame(df[(df["PW_SOC_CODE"]=="11-3021")&(df["CASE_STATUS"]==1)]["JOB_INFO_WORK_STATE"].value_counts()[:15]).reset_index()
succol.columns = ["SState","SCount"]
pd.concat([appcol,succol], axis = 1)

In [ ]:
jobcho = pd.DataFrame(df[(df["PW_SOC_CODE"]=="11-3021")&(df["CASE_STATUS"]==1)]["JOB_INFO_WORK_STATE"].value_counts()).reset_index()
jobcho.columns = ["SState","SCount"]
jobcho["text"] = jobcho["SState"].astype(str)+": "+jobcho["SCount"].astype(str)+" Successful Applications"
# Choropleth Map of EMPLOYER STATE
data = dict(type = "choropleth",
           locations = jobcho["SState"],
           locationmode = "USA-states",
           z = jobcho["SCount"].astype(float),
           text = jobcho["text"],
           colorbar={"title":"Applications"},
           colorscale = "oranges",
           marker = dict(line = dict(color = "rgb(255,255,255)")) )
layout = dict(title = "Successful 11-3021 Applications in States Worked In",
              geo = dict(scope = "usa",showlakes = True, lakecolor = "rgb(255,255,255)"))
choro = go.Figure(data, layout)
iplot(choro)

In [ ]:
# 11-3021, Computer and Information Systems Managers
# EMPLOYER NAME
appcol = pd.DataFrame(df[df["PW_SOC_CODE"]=="11-3021"]["EMPLOYER_NAME"].value_counts()[:15]).reset_index()
appcol.columns = ["AName","ACount"]
succol = pd.DataFrame(df[(df["PW_SOC_CODE"]=="11-3021")&(df["CASE_STATUS"]==1)]["EMPLOYER_NAME"].value_counts()[:15]).reset_index()
succol.columns = ["SName","SCount"]
pd.concat([appcol,succol], axis = 1)

In [ ]:
# 11-3021, Computer and Information Systems Managers
# Wage Offered
print("25th Percentile Application:",round(df[df["PW_SOC_CODE"]=="11-3021"]["WAGE_OFFER_FROM_9089"].quantile(.25),2))
print("25th Percentile Success:",round(df[(df["PW_SOC_CODE"]=="11-3021")&(df["CASE_STATUS"]==1)]["WAGE_OFFER_FROM_9089"].quantile(.25),2))
print("Mean Application:",round(df[df["PW_SOC_CODE"]=="11-3021"]["WAGE_OFFER_FROM_9089"].mean(),2))
print("Mean Success:",round(df[(df["PW_SOC_CODE"]=="11-3021")&(df["CASE_STATUS"]==1)]["WAGE_OFFER_FROM_9089"].mean(),2))
print("Median Application:",round(df[df["PW_SOC_CODE"]=="11-3021"]["WAGE_OFFER_FROM_9089"].median(),2))
print("Median Sucess:",round(df[(df["PW_SOC_CODE"]=="11-3021")&(df["CASE_STATUS"]==1)]["WAGE_OFFER_FROM_9089"].median(),2))
print("75th Percentile Application:",round(df[df["PW_SOC_CODE"]=="11-3021"]["WAGE_OFFER_FROM_9089"].quantile(.75),2))
print("75th Percentile Success:",round(df[(df["PW_SOC_CODE"]=="11-3021")&(df["CASE_STATUS"]==1)]["WAGE_OFFER_FROM_9089"].quantile(.75),2))

In [ ]:
# 11-3021, Computer and Information Systems Managers
# Class of Admission
appcol = pd.DataFrame(df[df["PW_SOC_CODE"]=="11-3021"]["CLASS_OF_ADMISSION"].value_counts()[:15]).reset_index()
appcol.columns = ["AClass","ACount"]
succol = pd.DataFrame(df[(df["PW_SOC_CODE"]=="11-3021")&(df["CASE_STATUS"]==1)]["CLASS_OF_ADMISSION"].value_counts()[:15]).reset_index()
succol.columns = ["SClass","SCount"]
pd.concat([appcol,succol], axis = 1)

In [ ]:
# 11-3021, Computer and Information Systems Managers
# Top Years
appcol = pd.DataFrame(df[df["PW_SOC_CODE"]=="11-3021"]["YEAR"].value_counts()[:15]).reset_index()
appcol.columns = ["AYear","ACount"]
succol = pd.DataFrame(df[(df["PW_SOC_CODE"]=="11-3021")&(df["CASE_STATUS"]==1)]["YEAR"].value_counts()[:15]).reset_index()
succol.columns = ["SYear","SCount"]
pd.concat([appcol,succol], axis = 1)

In [ ]:
# 11-3021, Computer and Information Systems Managers
# CORREL AND LIN REG BETWEEN YEARS AND APPLICATIONS


In [ ]:
### JOB LEVEL ANALYSIS ###

In [ ]:
# DISTRIBUTION OF JOB LEVEL OVER TIME
plt.figure(figsize=(12,10))
sns.countplot(x = "YEAR", hue = "PW_LEVEL_9089",data=df,hue_order=["LEVEL I","LEVEL II","LEVEL III","LEVEL IV"])

In [ ]:
# SUCCESS % OF JOB LEVEL IN AGGREGATE
lo = df[df["PW_LEVEL_9089"] == "LEVEL I"]["CASE_STATUS"].sum()/len(df[df["PW_LEVEL_9089"] == "LEVEL I"])
lt = df[df["PW_LEVEL_9089"] == "LEVEL II"]["CASE_STATUS"].sum()/len(df[df["PW_LEVEL_9089"] == "LEVEL II"])
lth = df[df["PW_LEVEL_9089"] == "LEVEL III"]["CASE_STATUS"].sum()/len(df[df["PW_LEVEL_9089"] == "LEVEL III"])
lf = df[df["PW_LEVEL_9089"] == "LEVEL IV"]["CASE_STATUS"].sum()/len(df[df["PW_LEVEL_9089"] == "LEVEL IV"])
#SIMPLE AVG
salw =(lo+lt+lth+lf)/4 
#WEIGHTED AVG
walw = (len(df[df["PW_LEVEL_9089"] == "LEVEL I"])*lo+
len(df[df["PW_LEVEL_9089"] == "LEVEL II"])*lt+
len(df[df["PW_LEVEL_9089"] == "LEVEL III"])*lth+
len(df[df["PW_LEVEL_9089"] == "LEVEL IV"])*lf)/len(df[df["PW_LEVEL_9089"].notna()])
#MERGING ALL
la = pd.DataFrame([lo,lt,lth,lf,salw,walw]).transpose()
la.columns=["LEVEL I","LEVEL II","LEVEL III","LEVEL IV","AVG","WAVG"]
la

In [ ]:
# GET A GROUPBY ON THE YEAR AND SEE EACH LEVELS SUCCESS AND APPLICATION COUNT
#PLOT THAT
# GET CORREL AND LIN REG OF THE SUCCESS AND APP COUNT OVER TIME FOR EACH LEVEL 

In [ ]:
# PW LEVEL DIST PER STATE WORKED IN
joblevel = pd.DataFrame(df.groupby("JOB_INFO_WORK_STATE")["PW_LEVEL_9089"].value_counts())
joblevel.columns = ["Count"]
joblevel = joblevel.reset_index()

In [ ]:
# LEVEL 1 DISTRIBUTION BY STATES WORKED IN
jobcho = joblevel[joblevel["PW_LEVEL_9089"]=="LEVEL I"]
jobcho.columns = ["State","Level","Count"]
jobcho["text"] = jobcho["State"].astype(str)+": "+jobcho["Count"].astype(str)+" Applications"
# Choropleth Map of EMPLOYER STATE
data = dict(type = "choropleth",
           locations = jobcho["State"],
           locationmode = "USA-states",
           z = jobcho["Count"].astype(float),
           text = jobcho["text"],
           colorbar={"title":"Applications"},
           colorscale = "oranges",
           marker = dict(line = dict(color = "rgb(255,255,255)")) )
layout = dict(title = "Level I Applications in States Worked In",
              geo = dict(scope = "usa",showlakes = True, lakecolor = "rgb(255,255,255)"))
choro = go.Figure(data, layout)
iplot(choro)

In [ ]:
# LEVEL 2 DISTRIBUTION BY STATES WORKED IN
jobcho = joblevel[joblevel["PW_LEVEL_9089"]=="LEVEL II"]
jobcho.columns = ["State","Level","Count"]
jobcho["text"] = jobcho["State"].astype(str)+": "+jobcho["Count"].astype(str)+" Applications"
# Choropleth Map of EMPLOYER STATE
data = dict(type = "choropleth",
           locations = jobcho["State"],
           locationmode = "USA-states",
           z = jobcho["Count"].astype(float),
           text = jobcho["text"],
           colorbar={"title":"Applications"},
           colorscale = "oranges",
           marker = dict(line = dict(color = "rgb(255,255,255)")) )
layout = dict(title = "Level II Applications in States Worked In",
              geo = dict(scope = "usa",showlakes = True, lakecolor = "rgb(255,255,255)"))
choro = go.Figure(data, layout)
iplot(choro)

In [ ]:
# LEVEL 3 DISTRIBUTION BY STATES WORKED IN
jobcho = joblevel[joblevel["PW_LEVEL_9089"]=="LEVEL III"]
jobcho.columns = ["State","Level","Count"]
jobcho["text"] = jobcho["State"].astype(str)+": "+jobcho["Count"].astype(str)+" Applications"
# Choropleth Map of EMPLOYER STATE
data = dict(type = "choropleth",
           locations = jobcho["State"],
           locationmode = "USA-states",
           z = jobcho["Count"].astype(float),
           text = jobcho["text"],
           colorbar={"title":"Applications"},
           colorscale = "oranges",
           marker = dict(line = dict(color = "rgb(255,255,255)")) )
layout = dict(title = "Level III Applications in States Worked In",
              geo = dict(scope = "usa",showlakes = True, lakecolor = "rgb(255,255,255)"))
choro = go.Figure(data, layout)
iplot(choro)

In [ ]:
# LEVEL 4 DISTRIBUTION BY STATES WORKED IN
jobcho = joblevel[joblevel["PW_LEVEL_9089"]=="LEVEL IV"]
jobcho.columns = ["State","Level","Count"]
jobcho["text"] = jobcho["State"].astype(str)+": "+jobcho["Count"].astype(str)+" Applications"
# Choropleth Map of EMPLOYER STATE
data = dict(type = "choropleth",
           locations = jobcho["State"],
           locationmode = "USA-states",
           z = jobcho["Count"].astype(float),
           text = jobcho["text"],
           colorbar={"title":"Applications"},
           colorscale = "oranges",
           marker = dict(line = dict(color = "rgb(255,255,255)")) )
layout = dict(title = "Level IV Applications in States Worked In",
              geo = dict(scope = "usa",showlakes = True, lakecolor = "rgb(255,255,255)"))
choro = go.Figure(data, layout)
iplot(choro)